# Stage 4: Rubric-Based Evaluation — Multi-Dimensional Quality Scoring

---

## 🎯 What You'll Learn

By the end of this walkthrough, you'll understand:

1. **What rubrics are** and why pass/fail isn't enough
2. **The four quality dimensions** and how to weight them
3. **How LLM-as-judge scoring works** behind the scenes
4. **How to set quality thresholds** to prevent regressions

---

## 📚 Understanding Rubric-Based Evaluation

### The Problem with Pass/Fail

Previous stages tell you **if** something works. But:
- A response can be "correct" but poorly written
- Quality can gradually degrade before hard failures
- Different use cases prioritize different qualities

### What Are Rubrics?

**Rubrics score responses across multiple dimensions**, giving you nuanced quality insights:

| Dimension | What It Measures | Example |
|-----------|-----------------|---------|
| **Relevance** (0-5) | Does it answer the question? | Off-topic = 1, Perfect = 5 |
| **Accuracy** (0-5) | Is the information correct? | Errors = 1, Verified = 5 |
| **Completeness** (0-5) | Does it cover everything? | Partial = 3, Full = 5 |
| **Clarity** (0-5) | Is it well-organized? | Confusing = 1, Clear = 5 |

### Why Rubrics Matter

```
Score Trend Over Time:
                        ⚠️ Quality degrading!
  5.0 ─────────────────────
      │ ████████
  4.0 │ ████████████████
      │ ████████████████████████
  3.0 │ ████████████████████████████████  ← Would pass binary tests
      │                                       but rubric shows decline
  2.0 │
      └──────────────────────────────────►
         v1.0   v1.1   v1.2   v1.3   time
```

---

## 🏗️ Where Rubrics Fit: The Eval Maturity Model

```
┌─────────────────────────────────────────────────────────────────┐
│                    EVAL FRAMEWORK MATURITY                      │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Stage 5: Experiments      ← Compare configurations            │
│      ▲                                                          │
│  ★ Stage 4: RUBRICS ★      ← Multi-dimensional scoring (HERE)  │
│      ▲                                                          │
│  Stage 3: Replay Harnesses ← Reproducibility (completed)       │
│      ▲                                                          │
│  Stage 2: Labeled Scenarios← Coverage mapping (completed)      │
│      ▲                                                          │
│  Stage 1: Golden Sets      ← Baseline correctness (completed)  │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

Rubrics add **gradual quality tracking** on top of the pass/fail foundations.

## 🔍 Behind the Scenes: How Rubric Scoring Works

Rubric scoring uses an **LLM-as-judge** pattern:

```
┌──────────────────────────────────────────────────────────────────────┐
│                    RUBRIC SCORING FLOW                                │
├──────────────────────────────────────────────────────────────────────┤
│                                                                       │
│  INPUT:                                                               │
│  ┌─────────────────────────────────────────────────────────────┐     │
│  │ Query:    "What is our PTO policy?"                         │     │
│  │ Response: "Employees receive 15 days of PTO annually..."    │     │
│  │ Sources:  ["pto_policy.md"]                                 │     │
│  └─────────────────────────────────────────────────────────────┘     │
│                            │                                          │
│                            ▼                                          │
│  LLM JUDGE (gpt-4o-mini):                                            │
│  ┌─────────────────────────────────────────────────────────────┐     │
│  │ "Score this response on each dimension (0-5):"              │     │
│  │                                                              │     │
│  │ - Relevance: Does it address the PTO question?              │     │
│  │ - Accuracy: Are the facts correct per the source?           │     │
│  │ - Completeness: Does it cover all PTO details?              │     │
│  │ - Clarity: Is it well-organized?                            │     │
│  └─────────────────────────────────────────────────────────────┘     │
│                            │                                          │
│                            ▼                                          │
│  OUTPUT:                                                              │
│  ┌─────────────────────────────────────────────────────────────┐     │
│  │ {                                                           │     │
│  │   "relevance": {"score": 5, "reason": "Directly answers"},  │     │
│  │   "accuracy": {"score": 4, "reason": "Correct, minor..."},  │     │
│  │   "completeness": {"score": 4, "reason": "Missing..."},     │     │
│  │   "clarity": {"score": 5, "reason": "Well structured"}      │     │
│  │ }                                                           │     │
│  └─────────────────────────────────────────────────────────────┘     │
│                            │                                          │
│                            ▼                                          │
│  WEIGHTED SCORE:                                                      │
│  ┌─────────────────────────────────────────────────────────────┐     │
│  │ relevance(5) × 0.25 + accuracy(4) × 0.35 +                  │     │
│  │ completeness(4) × 0.25 + clarity(5) × 0.15 = 4.40           │     │
│  │                                                              │     │
│  │ Quality Level: "Good" (3.5-4.4 range)                       │     │
│  └─────────────────────────────────────────────────────────────┘     │
│                                                                       │
└──────────────────────────────────────────────────────────────────────┘
```

**Key insight:** Weights can be customized per category (e.g., accuracy matters more for compliance questions).

## Setup

In [ ]:
import sys
from pathlib import Path

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent))

import yaml
from scorer import RubricScorer, print_result
from evaluator import run_rubric_evaluation, print_summary, load_rubric_test_cases

## 💻 Hands-On: Exploring Rubric Structure

Let's examine how rubrics are defined in YAML. Notice how each dimension has:
- A description of what it measures
- Specific criteria for each score level (0-5)
- A weight for the final calculation

In [ ]:
# Load and explore the rubrics
with open("rubrics.yaml") as f:
    rubrics = yaml.safe_load(f)

print("Evaluation Dimensions:")
print("=" * 50)
for dim_name, dim_info in rubrics["dimensions"].items():
    print(f"\n{dim_info['name']} (weight: {dim_info['weight']:.0%})")
    print(f"  {dim_info['description']}")

In [ ]:
# View the scoring criteria for one dimension
print("Accuracy Scoring Criteria:")
print("=" * 50)
for score, desc in rubrics["dimensions"]["accuracy"]["criteria"].items():
    print(f"  {score}: {desc}")

## 🧪 Scoring a Single Response

Let's run a query through the agent and see how the rubric scorer evaluates it. Watch for:
- Individual dimension scores
- Justifications from the judge
- The weighted overall score

In [ ]:
# Import the agent from local config
from rubric_config import ask_acme

# Run a query
query = "What is our remote work policy?"
response = ask_acme(query)

print(f"Query: {query}")
print("-" * 50)
print(f"Response:\n{response}")

In [ ]:
# Score the response
scorer = RubricScorer()
result = scorer.score(
    query=query,
    response=response,
    sources=["remote_work_policy.md"],
    category="policy"
)

print_result(result)

## ⚖️ Category-Specific Weights

**This is a powerful feature**: Different query types should prioritize different dimensions.

| Category | Higher Weight For | Why |
|----------|------------------|-----|
| **Policy** | Accuracy | Wrong policy info = compliance risk |
| **Metrics** | Completeness | Missing numbers = incomplete picture |
| **Engineering** | Clarity | Devs need clear instructions |

Let's see how weights differ by category:

In [ ]:
print("Default Weights vs Category-Specific Weights")
print("=" * 60)

default_weights = scorer.get_weights()
print(f"\nDefault:")
for dim, weight in default_weights.items():
    print(f"  {dim}: {weight:.0%}")

for category in ["policy", "metrics", "engineering"]:
    weights = scorer.get_weights(category)
    print(f"\n{category.capitalize()}:")
    for dim, weight in weights.items():
        diff = weight - default_weights[dim]
        indicator = "↑" if diff > 0 else "↓" if diff < 0 else " "
        print(f"  {dim}: {weight:.0%} {indicator}")

## 🏃 Running a Full Evaluation Suite

Let's run the complete rubric evaluation across multiple test cases. The summary will show:
- Quality distribution (Excellent/Good/Acceptable/Poor/Critical)
- Dimension averages (which dimensions are weakest?)
- Lowest scoring cases (where to focus improvement?)

In [ ]:
# Load test cases
test_cases = load_rubric_test_cases("rubrics.yaml")

print(f"Found {len(test_cases)} test cases:")
for case in test_cases[:5]:
    print(f"  [{case['id']}] {case['query'][:50]}...")

In [ ]:
# Run evaluation on a subset (for speed)
results, summary = run_rubric_evaluation(test_cases[:3], verbose=True)

In [ ]:
# View summary
print_summary(summary)

## 🚦 Quality Gates and Thresholds

**Quality gates** prevent regressions by failing builds when scores drop below thresholds.

| Score | Quality Level | Action |
|-------|--------------|--------|
| 4.5-5.0 | Excellent | Ship it! |
| 3.5-4.4 | Good | Minor improvements optional |
| 2.5-3.4 | Acceptable | Review before shipping |
| 1.5-2.4 | Poor | Must improve |
| 0-1.4 | Critical | Block deployment |

Let's implement quality gates:

In [ ]:
# Define minimum acceptable scores
QUALITY_THRESHOLDS = {
    "overall": 3.5,
    "accuracy": 4.0,  # Accuracy is critical
    "relevance": 3.5,
    "completeness": 3.0,
    "clarity": 3.0
}

def check_quality_gates(results):
    """Check if results meet quality thresholds."""
    failures = []
    
    # Check overall average
    avg_overall = sum(r.overall_score for r in results) / len(results)
    if avg_overall < QUALITY_THRESHOLDS["overall"]:
        failures.append(f"Overall avg {avg_overall:.2f} < {QUALITY_THRESHOLDS['overall']}")
    
    # Check dimension averages
    for dim in ["accuracy", "relevance", "completeness", "clarity"]:
        scores = [s.score for r in results for s in r.scores if s.dimension == dim]
        avg = sum(scores) / len(scores)
        if avg < QUALITY_THRESHOLDS[dim]:
            failures.append(f"{dim.capitalize()} avg {avg:.2f} < {QUALITY_THRESHOLDS[dim]}")
    
    return failures

# Check our results
failures = check_quality_gates(results)
if failures:
    print("Quality Gate Failures:")
    for f in failures:
        print(f"  - {f}")
else:
    print("All quality gates passed!")

## 🧪 Try Your Own Query

Test any query and see how it scores across all dimensions. Experiment with different:
- Query types (policy vs metrics)
- Categories (affects weights)
- Complexities

In [ ]:
# Change this to test different queries
test_query = "How do I request time off?"
test_category = "policy"  # optional: policy, metrics, engineering, cross_tool

# Get response
response = ask_acme(test_query)
print(f"Query: {test_query}")
print(f"Response: {response}\n")

# Score it
result = scorer.score(
    query=test_query,
    response=response,
    category=test_category
)
print_result(result)

## 🎓 Key Takeaways

1. **Rubrics catch gradual decline** — Quality scores reveal degradation before hard failures
2. **Four dimensions** — Relevance, Accuracy, Completeness, Clarity
3. **Weighted scoring** — Customize weights per category (accuracy > clarity for compliance)
4. **LLM-as-judge** — Use a model to evaluate another model's output
5. **Quality gates** — Set thresholds to prevent regressions in CI

---

## 📊 Interpreting Rubric Results

| What You See | What It Means | What To Do |
|-------------|---------------|------------|
| Low Relevance | Agent misunderstanding queries | Improve routing or system prompt |
| Low Accuracy | Facts wrong or hallucinated | Better retrieval or source grounding |
| Low Completeness | Missing parts of answer | Improve prompting for thoroughness |
| Low Clarity | Confusing responses | Format instructions in system prompt |

---

## ⏭️ What's Next?

Rubrics tell you quality scores, but how do you improve them? You need to:
- Compare different configurations (models, prompts, temperatures)
- Make data-driven decisions about what works best
- Track improvements over time

**Stage 5: Experiments** lets you systematically compare agent variants:

```bash
cd ../stage_5_experiments
uv run jupyter notebook walkthrough.ipynb
```